In [2]:
import pandas as pd 
import seaborn as sn 
import matplotlib.pyplot as plt 
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Load Dataset

In [3]:
df_full_feature = pd.read_csv('../Dataset/Standarisasi/processed_dataset_10_std.csv',sep=',')
df_chi_square = pd.read_csv('../Dataset/Standarisasi/processed_dataset_chi_square_std.csv', sep=',')
df_feature_selection = pd.read_csv('../Dataset/Standarisasi/processed_dataset_5_std.csv',sep=',')
df_slice = pd.read_csv('../Dataset/Standarisasi/processed_dataset_slice_std.csv',sep=',')

### Splitting Data

In [4]:
# Dataset Full Feature
full_feature_x = df_full_feature[df_full_feature.columns[:9]]
full_feature_y = df_full_feature['label']

full_feature_x.head(5)

,src,dst,Protocol,pktrate,pktperflow,dur,dur_nsec,bytecount,pktcount
0,-1.333061,-0.029423,-1.097155,-1.165007,-1.162827,2.354770,-0.814311,-0.781169,-1.000332
1,-0.521867,0.163403,0.089775,1.548708,1.548762,0.227232,-1.249944,-0.537869,3.240156
2,0.127088,0.741880,-1.097155,-1.159696,-1.162650,1.779375,-0.785508,-0.781491,-1.003410
3,0.938282,0.934706,0.089775,0.587490,0.588241,-0.788588,0.578995,-0.739081,-0.393352
4,-1.170822,-0.029423,1.276704,0.290096,0.287957,-0.117295,0.424183,1.082301,0.660084


In [5]:
# Dataset Chi Square 
chi_square_x = df_chi_square[df_chi_square.columns[:5]]
chi_square_y = df_chi_square['label']

chi_square_x.head(5)

,dst,src,Protocol,pktcount,pktcount.1
0,-0.029423,-1.333061,-1.097155,-1.000332,-1.000332
1,0.163403,-0.521867,0.089775,3.240156,3.240156
2,0.741880,0.127088,-1.097155,-1.003410,-1.003410
3,0.934706,0.938282,0.089775,-0.393352,-0.393352
4,-0.029423,-1.170822,1.276704,0.660084,0.660084


In [6]:
# Dataset Feature Selection
feature_selection_x = df_feature_selection[df_feature_selection.columns[:5]]
feature_selection_y = df_feature_selection['label']

feature_selection_x.head(5)

,pktrate,pktperflow,Protocol,src,dst
0,-1.165007,-1.162827,-1.097155,-1.333061,-0.029423
1,1.548708,1.548762,0.089775,-0.521867,0.163403
2,-1.159696,-1.162650,-1.097155,0.127088,0.741880
3,0.587490,0.588241,0.089775,0.938282,0.934706
4,0.290096,0.287957,1.276704,-1.170822,-0.029423


In [7]:
# Dataset Slice
slice_x = df_slice[df_slice.columns[:4]]
slice_y= df_slice['label']

slice_x.head(5)

,dst,src,Protocol,bytecount
0,-0.029423,-1.333061,-1.097155,-0.781169
1,0.163403,-0.521867,0.089775,-0.537869
2,0.741880,0.127088,-1.097155,-0.781491
3,0.934706,0.938282,0.089775,-0.739081
4,-0.029423,-1.170822,1.276704,1.082301


### Full Model

In [14]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(full_feature_x,full_feature_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape


(50540, 9, 1)

In [15]:
model_full = Sequential()
model_full.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_full.add(Dropout(0.5))
model_full.add(LSTM(32, return_sequences=False))
model_full.add(Dense(10, activation='relu'))
model_full.add(Dense(3, activation='softmax'))

model_full.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_full.fit(x_train, y_train, epochs=10)


Epoch 1/10
1580/1580 [==============================] - 19s 11ms/step - loss: 0.5047 - accuracy: 0.7522 - mean_squared_error: 0.3477 - mean_absolute_error: 0.4636
Epoch 2/10
1580/1580 [==============================] - 17s 11ms/step - loss: 0.1817 - accuracy: 0.9190 - mean_squared_error: 0.4267 - mean_absolute_error: 0.4644
Epoch 3/10
1580/1580 [==============================] - 18s 11ms/step - loss: 0.1521 - accuracy: 0.9331 - mean_squared_error: 0.4332 - mean_absolute_error: 0.4640
Epoch 4/10
1580/1580 [==============================] - 18s 11ms/step - loss: 0.1293 - accuracy: 0.9416 - mean_squared_error: 0.4374 - mean_absolute_error: 0.4639
Epoch 5/10
1580/1580 [==============================] - 19s 12ms/step - loss: 0.1191 - accuracy: 0.9486 - mean_squared_error: 0.4391 - mean_absolute_error: 0.4632 - ETA: 4s - loss: 
Epoch 6/10
1580/1580 [==============================] - 18s 12ms/step - loss: 0.1097 - accuracy: 0.9526 - mean_squared_error: 0.4411 - mean_absolute_error: 0.4635
Epo

In [16]:
y_pred_train = model_full.predict(x_train)
y_pred_val = model_full.predict(x_test)

In [17]:
y_pred_train = [np.argmax(element) for element in y_pred_train]
cm = confusion_matrix(y_train, y_pred_train, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)


[[18986   765]
 [  997 29792]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     30789
           1       0.95      0.96      0.96     19751

    accuracy                           0.97     50540
   macro avg       0.96      0.96      0.96     50540
weighted avg       0.97      0.97      0.97     50540

Accuracy: 96.51365255243371
Precision 95.01075914527348
False Negative Rate 3.8282540159135263


In [18]:
y_pred_val = [np.argmax(element) for element in y_pred_val]
cm = confusion_matrix(y_test, y_pred_val, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_test, y_pred_val))
print("Accuracy:",accuracy_score(y_test, y_pred_val)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)

[[ 8103   328]
 [  474 12756]]
              precision    recall  f1-score   support

           0       0.97      0.96      0.97     13230
           1       0.94      0.96      0.95      8431

    accuracy                           0.96     21661
   macro avg       0.96      0.96      0.96     21661
weighted avg       0.96      0.96      0.96     21661

Accuracy: 96.29749319052675
Precision 94.4735921650927
False Negative Rate 3.8241809490497847


### Chi Square Model

In [19]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(chi_square_x,chi_square_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape

(50540, 5, 1)

In [20]:
model_chi = Sequential()
model_chi.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_chi.add(Dropout(0.5))
model_chi.add(LSTM(32, return_sequences=False))
model_chi.add(Dense(10, activation='relu'))
model_chi.add(Dense(3, activation='softmax'))

model_chi.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_chi.fit(x_train, y_train, epochs=10)


Epoch 1/10
1580/1580 [==============================] - 17s 9ms/step - loss: 0.5680 - accuracy: 0.6788 - mean_squared_error: 0.3310 - mean_absolute_error: 0.4630
Epoch 2/10
1580/1580 [==============================] - 15s 9ms/step - loss: 0.4254 - accuracy: 0.7459 - mean_squared_error: 0.3641 - mean_absolute_error: 0.4632
Epoch 3/10
1580/1580 [==============================] - 15s 9ms/step - loss: 0.4155 - accuracy: 0.7582 - mean_squared_error: 0.3673 - mean_absolute_error: 0.4634
Epoch 4/10
1580/1580 [==============================] - 15s 10ms/step - loss: 0.4020 - accuracy: 0.7681 - mean_squared_error: 0.3708 - mean_absolute_error: 0.4638
Epoch 5/10
1580/1580 [==============================] - 16s 10ms/step - loss: 0.3946 - accuracy: 0.7739 - mean_squared_error: 0.3728 - mean_absolute_error: 0.4634
Epoch 6/10
1580/1580 [==============================] - 16s 10ms/step - loss: 0.3814 - accuracy: 0.7839 - mean_squared_error: 0.3766 - mean_absolute_error: 0.4644
Epoch 7/10
1580/1580 [===

In [21]:
y_pred_train = model_chi.predict(x_train)
y_pred_test = model_chi.predict(x_test)

In [22]:
y_pred_train = [np.argmax(element) for element in y_pred_train]
cm = confusion_matrix(y_train, y_pred_train, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)


[[14535  5216]
 [ 3865 26924]]
              precision    recall  f1-score   support

           0       0.84      0.87      0.86     30789
           1       0.79      0.74      0.76     19751

    accuracy                           0.82     50540
   macro avg       0.81      0.81      0.81     50540
weighted avg       0.82      0.82      0.82     50540

Accuracy: 82.03205381875742
Precision 78.9945652173913
False Negative Rate 28.347826086956523


In [23]:
y_pred_val = [np.argmax(element) for element in y_pred_val]
cm = confusion_matrix(y_test, y_pred_val, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_test, y_pred_val))
print("Accuracy:",accuracy_score(y_test, y_pred_val)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)

[[    0  8431]
 [    0 13230]]
              precision    recall  f1-score   support

           0       0.61      1.00      0.76     13230
           1       0.00      0.00      0.00      8431

    accuracy                           0.61     21661
   macro avg       0.31      0.50      0.38     21661
weighted avg       0.37      0.61      0.46     21661

Accuracy: 61.07751258021329
Precision nan
False Negative Rate inf


c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in longlong_scalars
  


### Feature Selection Model

In [8]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(feature_selection_x,feature_selection_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape

(50540, 5, 1)

In [9]:
model_feature = Sequential()
model_feature.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_feature.add(Dropout(0.5))
model_feature.add(LSTM(32, return_sequences=False))
model_feature.add(Dense(10, activation='relu'))
model_feature.add(Dense(3, activation='softmax'))

model_feature.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_feature.fit(x_train, y_train, epochs=10)


Epoch 1/10
1580/1580 [==============================] - 16s 8ms/step - loss: 0.5388 - accuracy: 0.7859 - mean_squared_error: 0.3475 - mean_absolute_error: 0.4627
Epoch 2/10
1580/1580 [==============================] - 12s 8ms/step - loss: 0.2794 - accuracy: 0.8947 - mean_squared_error: 0.4074 - mean_absolute_error: 0.4643
Epoch 3/10
1580/1580 [==============================] - 12s 8ms/step - loss: 0.2364 - accuracy: 0.9089 - mean_squared_error: 0.4165 - mean_absolute_error: 0.4630
Epoch 4/10
1580/1580 [==============================] - 12s 8ms/step - loss: 0.2258 - accuracy: 0.9105 - mean_squared_error: 0.4186 - mean_absolute_error: 0.4629
Epoch 5/10
1580/1580 [==============================] - 12s 7ms/step - loss: 0.2190 - accuracy: 0.9110 - mean_squared_error: 0.4202 - mean_absolute_error: 0.4638
Epoch 6/10
1580/1580 [==============================] - 12s 7ms/step - loss: 0.2151 - accuracy: 0.9150 - mean_squared_error: 0.4203 - mean_absolute_error: 0.4629
Epoch 7/10
1580/1580 [======

In [10]:
y_pred_train = model_feature.predict(x_train)
y_pred_val = model_feature.predict(x_test)

In [11]:
def calculate_precision(TP, FP):
    return TP/(TP+FP)


def calculate_fnr(TP, FP, FN):
    return FN/(TP+FP)
    

In [12]:
y_pred_train = [np.argmax(element) for element in y_pred_train]
cm = confusion_matrix(y_train, y_pred_train, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)


[[17680  2071]
 [ 1803 28986]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94     30789
           1       0.91      0.90      0.90     19751

    accuracy                           0.92     50540
   macro avg       0.92      0.92      0.92     50540
weighted avg       0.92      0.92      0.92     50540

Accuracy: 92.33478432924417
Precision 90.74577837088744
False Negative Rate 10.629779808037776


In [13]:
y_pred_val = [np.argmax(element) for element in y_pred_val]
cm = confusion_matrix(y_test, y_pred_val, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_test, y_pred_val))
print("Accuracy:",accuracy_score(y_test, y_pred_val)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)

[[ 7566   865]
 [  814 12416]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94     13230
           1       0.90      0.90      0.90      8431

    accuracy                           0.92     21661
   macro avg       0.92      0.92      0.92     21661
weighted avg       0.92      0.92      0.92     21661

Accuracy: 92.24874197867135
Precision 90.28639618138425
False Negative Rate 10.322195704057279


### Slice Model

In [28]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(slice_x,slice_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape

(50540, 4, 1)

In [29]:
model_slice = Sequential()
model_slice.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_slice.add(Dropout(0.5))
model_slice.add(LSTM(32, return_sequences=False))
model_slice.add(Dense(10, activation='relu'))
model_slice.add(Dense(2, activation='softmax'))
#model_slice.add(Dense(3, activation='softmax'))

model_slice.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_slice.fit(x_train, y_train, epochs=50)


Epoch 1/50
1580/1580 [==============================] - 13s 7ms/step - loss: 0.5195 - accuracy: 0.6867 - mean_squared_error: 0.3138 - mean_absolute_error: 0.5000
Epoch 2/50
1580/1580 [==============================] - 12s 7ms/step - loss: 0.3920 - accuracy: 0.7733 - mean_squared_error: 0.3639 - mean_absolute_error: 0.5000
Epoch 3/50
1580/1580 [==============================] - 11s 7ms/step - loss: 0.3792 - accuracy: 0.7826 - mean_squared_error: 0.3690 - mean_absolute_error: 0.5000
Epoch 4/50
1580/1580 [==============================] - 12s 7ms/step - loss: 0.3646 - accuracy: 0.7924 - mean_squared_error: 0.3747 - mean_absolute_error: 0.5000
Epoch 5/50
1580/1580 [==============================] - 13s 8ms/step - loss: 0.3605 - accuracy: 0.7980 - mean_squared_error: 0.3765 - mean_absolute_error: 0.5000
Epoch 6/50
1580/1580 [==============================] - 13s 8ms/step - loss: 0.3522 - accuracy: 0.8018 - mean_squared_error: 0.3798 - mean_absolute_error: 0.5000
Epoch 7/50
1580/1580 [======

In [30]:
y_pred_train = model_slice.predict(x_train)
y_pred_val = model_slice.predict(x_test)

In [31]:
def calculate_precision(TP, FP):
    return TP/(TP+FP)


def calculate_fnr(TP, FP, FN):
    return FN/(TP+FP)
    

In [32]:
y_pred_train = [np.argmax(element) for element in y_pred_train]
cm = confusion_matrix(y_train, y_pred_train, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)


[[17052  2699]
 [ 2903 27886]]
              precision    recall  f1-score   support

           0       0.91      0.91      0.91     30789
           1       0.85      0.86      0.86     19751

    accuracy                           0.89     50540
   macro avg       0.88      0.88      0.88     50540
weighted avg       0.89      0.89      0.89     50540

Accuracy: 88.91571032845272
Precision 85.45226760210474
False Negative Rate 13.525432222500628


In [33]:
y_pred_val = [np.argmax(element) for element in y_pred_val]
cm = confusion_matrix(y_test, y_pred_val, labels=[1,0])
TP = cm[0][0]
FP = cm[1][0]
FN = cm[0][1]
TN = cm[1][1]

print(cm)
print(classification_report(y_test, y_pred_val))
print("Accuracy:",accuracy_score(y_test, y_pred_val)*100)
print("Precision",(calculate_precision(TP,FP))*100)
print("False Negative Rate", (calculate_fnr(TP,FP,FN))*100)

[[ 7199  1232]
 [ 1350 11880]]
              precision    recall  f1-score   support

           0       0.91      0.90      0.90     13230
           1       0.84      0.85      0.85      8431

    accuracy                           0.88     21661
   macro avg       0.87      0.88      0.87     21661
weighted avg       0.88      0.88      0.88     21661

Accuracy: 88.07995937399012
Precision 84.208679377705
False Negative Rate 14.411042227161072
